In [9]:
!pip install networkx

In [10]:
import sys

MODULES_PATH = "../../modules"
if MODULES_PATH not in sys.path:
    sys.path.append(MODULES_PATH)

In [11]:
import networkx as nx
import pickle
import heapq
from collections import defaultdict
import datetime
import copy
from collections import deque
import pandas as pd
import scipy.stats as stats

from hive_wrapper import *
from database import *
from utils import *
from route_planner import *
from validation_utils import *
from visualization import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Load the graph from a pickle file
with open("new_graph.pkl", "rb") as f:
    graph = pickle.load(f)

In [13]:
class RoutePlanner:
    """
    Finds the shortest path between two nodes in a network graph.
    Each edge has departure and arrival times which means all edges are not compatible if their timings aren't compatible.
    Use Dijkstra's algorithm to find the shortest path.
    """
    def __init__(self, 
                 network_graph, 
                 source_node, 
                 target_node,
                 deadline,
                 database
                ):
        self.G = network_graph
        self.source = source_node
        self.target = target_node
        self.deadline = deadline
        
        self.delay_database = database
        self.day_periods = {
            "morning": ("06:00", "08:30"),
            "prenoon": ("08:30", "12:00"),
            "afternoon": ("12:00", "15:00"),
            "latenoon": ("15:00", "18:00"),
            "evening": ("18:00", "22:00")
        }
        
    def extract_path(self, node, previous_edge):
        res = []
        while node:
            if node in previous_edge:
                res.append(previous_edge[node])
                node = previous_edge[node][0]
            else:
                node = None
        return res
        
    def extract_trips(self, path):
        trips = set()
        for edge in path:
            trips.add(edge[2]['trip_id'])
        return trips
        
    def find_paths(self, n=1):
        n = max(1, n)
        res = []
        banned_trips_queue = deque()
        banned_trips_queue.append(set())
        while n > 0 and banned_trips_queue:
            bt = banned_trips_queue.popleft() 
            # print(bt)
            node, previous_edge = self.dijkstra_algorithm(banned_trips=bt)
            if node:
                n -= 1
                path = self.extract_path(node, previous_edge)
                res.append(path)
                trips = self.extract_trips(path)
                for t in trips:
                    if t:
                        banned_trips_queue.append(set(list(bt) + [t]))
                        
        # compute success probability
        routes_with_success_proba = []
        for route in res:
            routes_with_success_proba.append((route, self.compute_success_probability(route)))
        
        return routes_with_success_proba
    
    def compute_success_probability(self, route):
        """Computes the probability of success of the route.

        Returns:
            p_success (float): the probability of success of the route.
        """                        
        def transfer_success_probability(edge_data):
            assert edge_data is not None
            assert edge_data["next_departure_time"] is not None
            assert edge_data["next_arrival_time"] is not None
            
            # convert strings to datetime
            next_departure_time = pd.to_datetime(edge_data["next_departure_time"], format="%H:%M:%S")
            next_arrival_time = pd.to_datetime(edge_data["next_arrival_time"], format="%H:%M:%S")
            
            # determine day period
            formatted_arrival_time = next_arrival_time.strftime("%H:%M")
            if formatted_arrival_time < self.day_periods["morning"][1]:
                day_period = "morning"
            elif formatted_arrival_time < self.day_periods["prenoon"][1]:
                day_period = "prenoon"
            elif formatted_arrival_time < self.day_periods["afternoon"][1]:
                day_period = "afternoon"
            elif formatted_arrival_time < self.day_periods["latenoon"][1]:
                day_period = "latenoon"
            else:
                day_period = "evening"
            
            # compute mean and std of the delay for the arrival stop of the edge
            delay_mean = self.delay_database.fetch_delay(
                metric="avg", stop_name=edge_data["stop_name"], day_period=day_period,
                transport_type=edge_data["transport_type"], transport_subtype=edge_data["transport_subtype"]
            )
            delay_std = self.delay_database.fetch_delay(
                metric="std", stop_name=edge_data["stop_name"], day_period=day_period,
                transport_type=edge_data["transport_type"], transport_subtype=edge_data["transport_subtype"]
            )
            max_possible_delay_s = (next_departure_time - next_arrival_time).seconds
            
            # compute the probability of success of the transfer
            if delay_mean is None or delay_std is None:
                # we don't have an estimate for the given stop name
                assert delay_mean is None and delay_std is None, "either both mean and std are null or neither is."
                p_success = 1.0
            else:
                p_success = stats.norm.cdf(max_possible_delay_s, loc=delay_mean, scale=delay_std)
                
            return p_success
        
        # start at the first non-walkable edge
        current_trip_id = None
        i = 0
        for _, _, edge_data in route:
            current_trip_id = edge_data["trip_id"]
            if current_trip_id is not None:
                break
            i += 1
        
        # if there is no non-walkable edge, probability of success is 1
        if current_trip_id is None:
            # the route must contain only walkable edges
            for _, _, edge_data in route:
                assert edge_data["transport_type"] == "walk", f"{edge_data} must be a walkable edge since its trip_id is None."
            return 1.0

        # otherwise, the route has at least one non-walkable edge
        # compute route's probability of success 
        route_p_success = 1.0
        for _, _, edge_data in route[1:]:
            if edge_data["trip_id"] is None:
                # must be a walkable edge or else there is an issue
                assert edge_data["transport_type"] == "walk", f"{edge_data} must be a walkable edge since its trip_id is None."
                continue
            if (edge_data["next_departure_time"] is None) or (edge_data["next_arrival_time"] is None):
                # next edge is walkable, no way to check that aside from:
                assert (edge_data["next_departure_time"] is None) and (edge_data["next_arrival_time"] is None), "both next_departure_time and next_arrival_time must be None or neither is."
                continue
            # otherwise, if the edge is a transfer, compute its probability of success
            is_transfer = (current_trip_id != edge_data["trip_id"])
            if is_transfer:
                route_p_success *= transfer_success_probability(edge_data)
                current_trip_id = edge_data["trip_id"]
        
        return route_p_success
    
    def dijkstra_algorithm(self, transfer_time=120, banned_trips=set()):
        """
        Returns the shortest path between source and target nodes.
        """

        distance = defaultdict(lambda: float('inf'))
        walking_time = defaultdict(lambda: float('inf'))
        number_of_transfers = defaultdict(lambda: float('inf'))
        distance[self.target] = 0
        walking_time[self.target] = 0
        number_of_transfers[self.target] = 0
        previous_edge = {}
        not_visited = set(self.G.nodes())
        not_visited.remove(self.target)
        path_found = False

        edges = self.G.edges(self.target, data=True)

        for edge in edges:
            edge_data = edge[2]
            if edge_data['trip_id'] in banned_trips:
                continue
            if edge_data['is_walkable'] or (datetime.datetime.strptime(edge_data['next_arrival_time'], '%H:%M:%S') <= self.deadline):
                # add the edge to the path
                w = 0
                if edge_data['is_walkable'] == True:
                    edge_data['departure_time'] = (self.deadline - datetime.timedelta(seconds=int(edge_data['duration_s']))).strftime("%H:%M:%S")
                    w += edge_data['duration_s']
 
                d = (self.deadline - datetime.datetime.strptime(edge_data['departure_time'], '%H:%M:%S')).total_seconds()
                
                if d < distance[edge[1]] or (d == distance[edge[1]] and w < walking_time[edge[1]]):
                    distance[edge[1]] = d
                    walking_time[edge[1]] = w
                    number_of_transfers[edge[1]] = 0
                    previous_edge[edge[1]] = edge
    
                if edge[1] == self.source:
                    path_found = True
        
        if path_found:
            return self.source, previous_edge 
        
        for i in range(self.G.number_of_nodes() - 1):
            u = min(not_visited, key=lambda x: distance[x])
            not_visited.remove(u)

            edges = self.G.edges(u, data=True)

            for edge in edges:
                edge_data = edge[2]
                if edge_data['trip_id'] in banned_trips:
                    continue

                prev_edge = previous_edge[u]
                if not (edge_data['is_walkable'] and prev_edge[2]['is_walkable']):
                    if edge_data['is_walkable'] or (edge_data['trip_id'] == prev_edge[2]['trip_id']):
                        w = 0
                        if edge_data['is_walkable'] == True:
                            edge[2]['departure_time'] = (datetime.datetime.strptime(prev_edge[2]['departure_time'],'%H:%M:%S') - datetime.timedelta(seconds=int(edge_data['duration_s']))).strftime("%H:%M:%S")
                            w += edge_data['duration_s']

                        # if we continue on the same trip
                        d = (datetime.datetime.strptime(prev_edge[2]['departure_time'], '%H:%M:%S') - datetime.datetime.strptime(edge_data['departure_time'], '%H:%M:%S')).total_seconds()

                        new_d = distance[u] + d
                        new_w = walking_time[u] + w
                        new_n = number_of_transfers[u]

                        if new_d < distance[edge[1]] or (new_d == distance[edge[1]] and new_w < walking_time[edge[1]]) or (new_d == distance[edge[1]] and new_w == walking_time[edge[1]] and new_n < number_of_transfers[edge[1]]):
                            distance[edge[1]] = new_d
                            walking_time[edge[1]] = new_w
                            number_of_transfers[edge[1]] = new_n
                            previous_edge[edge[1]] = edge
                            
                        if edge[1] == self.source:
                            path_found = True
                    else:
                        waiting_time = (datetime.datetime.strptime(prev_edge[2]['departure_time'], '%H:%M:%S') - datetime.timedelta(seconds=transfer_time)) - datetime.datetime.strptime(edge_data['next_arrival_time'], '%H:%M:%S')
                        waiting_seconds = waiting_time.total_seconds()
                        if waiting_seconds >= 0:
                            d = (datetime.datetime.strptime(prev_edge[2]['departure_time'], '%H:%M:%S') - datetime.datetime.strptime(edge_data['departure_time'], '%H:%M:%S')).total_seconds()

                            new_d = distance[u] + d
                            new_w = walking_time[u]
                            new_n = number_of_transfers[u] + 1

                            if new_d < distance[edge[1]] or (new_d == distance[edge[1]] and new_w < walking_time[edge[1]]) or (new_d == distance[edge[1]] and new_w == walking_time[edge[1]] and new_n < number_of_transfers[edge[1]]):
                                distance[edge[1]] = new_d
                                walking_time[edge[1]] = new_w
                                number_of_transfers[edge[1]] = new_n
                                previous_edge[edge[1]] = edge
                                
                            if edge[1] == self.source:
                                path_found = True
                                
            if path_found:
                return self.source, previous_edge
        return None, None
    

In [14]:
env_vars = get_env_vars()
hive_connection = HiveConnection(*env_vars)
db = Database(hive_connection)

avg_delay_test = db.fetch_delay(
    "avg", stop_name="zürich, leutschenbach", day_period="morning", transport_type=None, transport_subtype=None, force_fetch=True)

In [18]:
deadline = datetime.datetime.strptime('14:00:00', '%H:%M:%S')

rp = RoutePlanner(graph, "8572644", "8575929", deadline=deadline, database=db)

routes = rp.find_paths(5)

# print(len(routes))

for route in routes:
    for edge in route:
        print(edge)
    print()

set()
{'75.TA.92-640-j23-1.4.R'}
{'58.TA.96-165-4-j23-1.3.H'}
{'270.TA.91-14-C-j23-1.97.R'}
{'659.TA.91-24-j23-1.255.R'}
[('8572645', '8572644', {'trip_id': None, 'trip_headsign': None, 'route_name': None, 'transport_type': 'walk', 'transport_subtype': 'walk', 'stop_name': 'staffeln ag, gass', 'departure_time': '10:44:26', 'next_stop_name': 'hermetschwil, kloster', 'next_arrival_time': None, 'next_departure_time': None, 'is_walkable': True, 'duration_s': 454.0}), ('8572643', '8572645', {'trip_id': '53.TA.96-170-3-j23-1.4.R', 'trip_headsign': 'Bremgarten AG, Obertorplatz', 'route_name': '339', 'transport_type': 'bus', 'transport_subtype': 'b', 'stop_name': 'hermetschwil, kloster', 'departure_time': '10:52:00', 'next_stop_name': 'bremgarten ag, west', 'next_arrival_time': '10:54:00', 'next_departure_time': '10:55:00', 'is_walkable': False, 'duration_s': 120.0}), ('8502272:0:1/2', '8572643', {'trip_id': None, 'trip_headsign': None, 'route_name': None, 'transport_type': 'walk', 'transport_